# Purpose

This notebook takes Ecosystem Dashbaord dumps of Github event data and:
1. Merges them (as may want to query different data ranges or different database instances as we have IPFS, Filecoin, and libp2p instances)
2. Deduplicates them (as there is often depulicate events across the IPFS, Filecoin, and libp2p instances)
3. Cleans them up (as the github event types and actions aren't the most intuitive or concicse)
4. Adds additional data (tagging data with YYYYMM or YearQuarter is helpful for summary)
5. Exports to a more useful form of showing a monthly rollup of how many github actions a given actor took in a given month/repo.

This allows for easy import to do further analysis/summary in places like Google Sheets.  
@biglep has been publishing to https://docs.google.com/spreadsheets/d/1jR6ueqrcdg6CYUvV3ibVWMjkGKj5WlU8ysuCO0TrHvo/edit?usp=sharing

This data can be useful for getting insight into github activity in our repos.  @biglep has found this useful for:
1. understanding at a high level who some of our contributors are and how they're changing
2. getting a pulse at performance review time on where various team members have been contributing

# SQL Query
Ideally this notebook should do the notebook queries directly, but at least as of 2023-06-19, biglep@ used his previous Postgres connections setup for `pgAdmin` and took dumps from there.

The query being run was:

```sql
SELECT
    github_id,
    actor,
    event_type,
    action,
    org,
    repository_full_name,
    created_at,
    core,
    bot,
    pmf
FROM
    events
WHERE
    -- adjust the dates as needed
    created_at >= DATE '2023-01-01'
    AND created_at < DATE '2023-06-15'
    AND org IN (
        -- These are PL's "core" orgs
        'multiformats',
        'ipld',
        'libp2p',
        'ipfs',
        'ipfs-examples',
        'ipfs-shipyard',
        'ipfs-inactive',
        'ipfs-cluster',
        'ipni',
        'protocol',
        'web3-storage',
        'nftstorage',
        'ProtoSchool',
        'pl-strflt',
        'plprobelab',
        'application-research',
        'filecoin-project',
        'filecoin-shipyard',
        'testground'
    )
    AND event_type IN (
        -- https://docs.github.com/en/developers/webhooks-and-events/events/github-event-types
        'IssueCommentEvent',
        'IssuesEvent',
        'PullRequestEvent',
        'PullRequestReviewEvent',
        'PullRequestReviewCommentEvent',
        'ReleaseEvent'
    )
    AND actor NOT LIKE '%bot%'
    AND actor NOT LIKE '%codecov%';
```

# Code
The skeleton of this code was generated from ChatGPT and modified from there.
It assumes the .csv files from SQL dumps all live in an input directory.

In [ ]:
import pandas as pd
import os
import datetime
report_date = datetime.date.today()
report_date_str = report_date.strftime("%Y-%m-%d")

# Initialize an empty DataFrame
df = pd.DataFrame()

# Specify the directory paths for input and output
input_directory = 'data/ecosystem-dashboard-github-event-dumps-from-sql'
output_directory = f"{input_directory}/output"

# List all files in the directory
files = os.listdir(input_directory)

# Iterate over each file
for file_name in files:
    # Check if the item is a CSV file
    file_path = os.path.join(input_directory, file_name)
    if os.path.isfile(file_path) and file_path.endswith(".csv"):
        # Open the file
        print(f"Reading {file_name}")
        temp_df = pd.read_csv(file_path, index_col='github_id')
        print(f"Read {file_name} with {len(temp_df)} rows")
        df = pd.concat([df, temp_df])

# Deduplicate the rows based on the index (github_id)
print(f"Concatenated size: {len(df)} rows")
df = df[~df.index.duplicated(keep='first')]
print(f"Deduplicated size: {len(df)} rows")

# Convert 'created_at' column to datetime format
df['created_at'] = pd.to_datetime(df['created_at'])

# Add 'Year Month' column in 'YYYYMM' format
df['year_month'] = df['created_at'].dt.strftime('%Y%m')

# Add 'Year Quarter' column
df['year_quarter'] = df['created_at'].dt.to_period('Q')

# Create 'repository_name' column by extracting repository name
df['repository_name'] = df['repository_full_name'].str.split('/').str.get(1)

mapping_to_friendly_event_name = {
    "IssueCommentEvent-created" : "Issue Comment",
	"IssuesEvent-closed" : "Issue Close",
	"IssuesEvent-opened" : "Issue Open",
	"IssuesEvent-reopened" : "Issue Reopen",
	"PullRequestEvent-closed" : "PR Close",
	"PullRequestEvent-opened" : "PR Open",
	"PullRequestEvent-reopened" : "PR Reopen",
	"PullRequestReviewCommentEvent-created" : "PR Comment",
	"PullRequestReviewEvent-created" : "PR Review",
	"ReleaseEvent-published" : "Release Publish",
}

# Define a function to add a new column based on row values
# This is so get human friendly "event_type" + "action" strings
def get_friendly_event_name(row):
    # Access values of specific columns in the row
    friendly_name = row['event_type'] + "-" + row['action']
    return mapping_to_friendly_event_name.get(friendly_name, friendly_name)

# Apply the function to each row and assign the result to a new column
df['friendly_event_name'] = df.apply(lambda row: get_friendly_event_name(row), axis=1)

df.to_csv(f"{output_directory}/github-event-data-cleaned-{report_date_str}.csv")

# Print the resulting DataFrame
df

In [ ]:
summary_table = pd.pivot_table(df.reset_index(), 
                               index=['org', 'repository_name', 'actor', 'year_quarter', 'year_month', 'friendly_event_name'], 
                               values='github_id', 
                               aggfunc='count',
                               fill_value=0)

summary_table.rename(columns={'github_id': 'count'}, inplace=True)

summary_table

In [ ]:
summary_table.to_csv(f"{output_directory}/github-event-monthly-summary-{report_date_str}.csv")

In [ ]:
audit_table = pd.pivot_table(df.reset_index(), 
                               index=['year_month'], 
                               columns=['org'],
                               values='github_id', 
                               aggfunc='count',
                               fill_value=0)

audit_table

In [ ]:
audit_table.to_csv(f"{output_directory}/github-event-monthly-summary-audit-{report_date_str}.csv")